# Community FBA models

In [1]:
using COBREXA

Here we will construct a community FBA model of two  *E. coli* "core" models
that can interact by exchanging selected metabolites. To do this, we will need
the model, which we can download if it is not already present.

In [2]:
download_model(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
    "7bedec10576cfe935b19218dc881f3fb14f890a1871448fc19a9b4ee15b448d8",
)

[ Info: using cached `e_coli_core.json'


"e_coli_core.json"

Additionally to COBREXA and the model format package, we will need a solver
-- let's use GLPK here:

In [3]:
import JSONFBCModels
import GLPK
import AbstractFBCModels.CanonicalModel as CM

ecoli1 = load_model("e_coli_core.json", CM.Model)
ecoli1.reactions["EX_glc__D_e"].lower_bound = -1000.0
ecoli1.reactions["EX_glc__D_e"].upper_bound = 1000.0
ecoli2 = deepcopy(ecoli1)

AbstractFBCModels.CanonicalModel.Model(
  reactions = Dict{String, AbstractFBCModels.CanonicalModel.Reaction}("ACALD" =…
  metabolites = Dict{String, AbstractFBCModels.CanonicalModel.Metabolite}("co2_…
  genes = Dict{String, AbstractFBCModels.CanonicalModel.Gene}("b4301" => Abstra…
)


customize models a bit

In [4]:
ecoli1.reactions["CYTBD"].lower_bound = ecoli1.reactions["CYTBD"].upper_bound = 0.0
ecoli2.reactions["FBA"].lower_bound = ecoli2.reactions["FBA"].upper_bound = 0.0

0.0

solve

In [5]:
solution = community_flux_balance_analysis(
    ["bug1" => (ecoli1, 0.2), "bug2" => (ecoli2, 0.8)],
    ["EX_glc__D_e" => (-10.0, 0.0)],
    optimizer = GLPK.Optimizer,
)

ConstraintTrees.Tree{Float64} with 6 elements:
  :bug1                => ConstraintTrees.Tree{Float64}(#= 4 elements =#)
  :bug2                => ConstraintTrees.Tree{Float64}(#= 4 elements =#)
  :community_balance   => ConstraintTrees.Tree{Float64}(#= 20 elements =#)
  :community_biomass   => 0.523716
  :community_exchanges => ConstraintTrees.Tree{Float64}(#= 20 elements =#)
  :equal_growth        => ConstraintTrees.Tree{Float64}(#= 1 element =#)

## Compare

We can now e.g. observe the differences in individual pairs of exchanges:

In [6]:
import ConstraintTrees as C

C.zip(
    tuple,
    solution.bug1.interface.exchanges,
    solution.bug2.interface.exchanges,
    Tuple{Float64,Float64},
)

ConstraintTrees.Tree{Tuple{Float64, Float64}} with 20 elements:
  :EX_ac_e     => (16.5424, 0.0)
  :EX_acald_e  => (0.0, 0.0)
  :EX_akg_e    => (0.0, 0.0)
  :EX_co2_e    => (-0.935723, 19.8177)
  :EX_etoh_e   => (15.9879, 0.0)
  :EX_for_e    => (35.0581, 2.52782)
  :EX_fru_e    => (0.0, 0.0)
  :EX_fum_e    => (0.0, 0.0)
  :EX_glc__D_e => (-20.245, -7.43875)
  :EX_gln__L_e => (0.0, 0.0)
  :EX_glu__L_e => (8.88178e-16, 0.0)
  :EX_h2o_e    => (-13.1086, 23.6327)
  :EX_h_e      => (62.1062, 13.0336)
  :EX_lac__D_e => (0.0, 0.0)
  :EX_mal__L_e => (0.0, 0.0)
  :EX_nh4_e    => (-2.85572, -2.85572)
  :EX_o2_e     => (0.0, -20.4762)
  :EX_pi_e     => (-1.92659, -1.92659)
  :EX_pyr_e    => (0.0, 0.0)
  :EX_succ_e   => (7.10543e-15, 0.0)

## Which composition is best?

In [7]:
screen(0.0:0.1:1.0) do ratio2
    ratio1 = 1 - ratio2
    res = community_flux_balance_analysis(
        ["bug1" => (ecoli1, ratio1), "bug2" => (ecoli2, ratio2)],
        ["EX_glc__D_e" => (-10.0, 0.0)],
        interface = :sbo, # usually more reproducible
        optimizer = GLPK.Optimizer,
    )
    (ratio1, ratio2) => (isnothing(res) ? nothing : res.community_biomass)
end

11-element Vector{Pair{Tuple{Float64, Float64}, Float64}}:
                 (1.0, 0.0) => 0.21166294973531166
                 (0.9, 0.1) => 0.23424604629802845
                 (0.8, 0.2) => 0.25971971809640987
                 (0.7, 0.3) => 0.2886768902449847
                 (0.6, 0.4) => 0.32188455339556993
                 (0.5, 0.5) => 0.36035269877962506
                 (0.4, 0.6) => 0.4054388316161014
 (0.30000000000000004, 0.7) => 0.4590115020201965
 (0.19999999999999996, 0.8) => 0.5237157737585179
 (0.09999999999999998, 0.9) => 0.6034233598466001
                 (0.0, 1.0) => 0.7040369478590228

...seems a lot like `bug1` will eventually disappear.

## Note about interfaces

Some work:

In [8]:
flux_balance_constraints(ecoli1, interface = :sbo).interface

ConstraintTrees.Tree{ConstraintTrees.Constraint} with 2 elements:
  :biomass   => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 1 element =…
  :exchanges => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 20 elements…

Some generally don't do well:

In [9]:
flux_balance_constraints(ecoli1, interface = :boundary).interface

ConstraintTrees.Tree{ConstraintTrees.Constraint} with 1 element:
  :boundary => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 20 elements …

# TODO old version

Community models work by joining its members together through their exchange
reactions, weighted by the abundance of each microbe. These exchange reactions
are then linked to an environmental exchange. For more theoretical details,
see "Gottstein, et al, 2016, Constraint-based stoichiometric modelling from
single organisms to microbial communities, Journal of the Royal Society
Interface".

## Building a community of two *E. coli*s

Here we will construct a simple community of two interacting microbes. To do
this, we need to import the models. We will represent the models only as
constraint trees, because it is easier to build the model explicitly than
rely on an opaque one-shot function.

ecoli1 = flux_balance_constraints(model, interface = :sbo)

In [10]:
#ecoli2 = flux_balance_constraints(model, interface = :sbo)

Since the models are usually used in a mono-culture context, the glucose input
for each individual member is limited. We need to undo this limitation, and
rather rely on the constrained environmental exchange reaction (and the bounds
we set for it earlier).

In [11]:
#ecoli1.fluxes.EX_glc__D_e.bound = C.Between(-1000.0, 1000.0)
#ecoli2.fluxes.EX_glc__D_e.bound = C.Between(-1000.0, 1000.0)

To make the community interesting, we can limit different reactions in both
members to see how the  models cope together:

In [12]:
#ecoli1.fluxes.CYTBD.bound = C.Between(-10.0, 10.0)
#ecoli2.fluxes.ACALD.bound = C.Between(-5.0, 5.0)

Because we created the trees with interfaces, we can connect them easily to
form a new model with the interface. For simplicity, we use the
interface-scaling functionality of [`interface_constraints`](@ref
COBREXA.interface_constraints) to bring in cFBA-like community member
abundances:

In [13]:
#cc = interface_constraints(
#:bug1 => (ecoli1, ecoli1.interface, 0.2),
#:bug2 => (ecoli2, ecoli2.interface, 0.8),
#)

To make the community behave as expected, we need to force equal (scaled)
growth of all members:

In [14]:
#cc *=
#:equal_growth^equal_value_constraint(
#cc.bug1.fluxes.BIOMASS_Ecoli_core_w_GAM,
#cc.bug2.fluxes.BIOMASS_Ecoli_core_w_GAM,
#)

Now we can simulate the community growth by optimizing the new "interfaced"
biomass:

In [15]:
#optimized_cc = optimized_values(
#cc,
#objective = cc.interface.biomass.BIOMASS_Ecoli_core_w_GAM.value,
#optimizer = GLPK.Optimizer,
#)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*